In [ ]:
from bertopic import BERTopic
from flair.embeddings import TransformerDocumentEmbeddings
import os
from os.path import join
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from umap import UMAP
import warnings
warnings.filterwarnings('ignore')

from nltk.corpus import stopwords
stopwords_nl = stopwords.words('dutch')
stopwords_nl = stopwords_nl + ['2016','2017','2018','2019','2020','2021','2022','50','we','ten','opzichte','daardoor','for','verder','ervoor','daardoor','per','vooral']

In [ ]:
OUTPUT_DATA_PATH = join('..','data','output','')
MODEL_PATH = join('..','data','models','Bertopic','')

### Own data

In [ ]:
#Import sentences
df = pd.read_excel(OUTPUT_DATA_PATH + 'df_results_mvb_big_sen_pred.xlsx',index_col=0)
df = df[df.prediction==1]

In [ ]:
docs = []
for sen in df['dnb_nlp:formatted']:
    docs.append(sen)

In [ ]:
# The same embedding model needs to be used for both topic models
# and since we are dealing with multiple languages, the model needs to be multi-lingual
sentence_model = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")
# To make this example reproducible
umap_model = UMAP(n_neighbors=200, n_components=5, 
                  min_dist=0.0, metric='cosine', random_state=42)

# Dutch
vectorizer_model = CountVectorizer(stop_words=stopwords_nl)
topic_model = BERTopic(embedding_model=sentence_model,umap_model=umap_model,vectorizer_model=vectorizer_model,min_topic_size=25)
topics, probs = topic_model.fit_transform(docs)

topic_model.save(MODEL_PATH + "bertopic_model_mvb_new") #save topic model
topic_model.get_topic_info()

In [ ]:
topic_model.reduce_topics(docs, topics, nr_topics=50)
topic_model.save(MODEL_PATH + "bertopic_model_mvb_50topics_new") #save topic model
topic_model.get_topic_info()

In [ ]:
topic_model.reduce_topics(docs, topics, nr_topics=25)
topic_model.save(MODEL_PATH + "bertopic_model_mvb_reducedtopics_new") #save topic model
topic_model.get_topic_info()

In [ ]:
# #topic_model = BERTopic.load(MODEL_PATH + "bertopic_model_mvb_new") #load topic model

In [ ]:
topic_model.get_topics().keys()

In [ ]:
# #Visualize topic model
# #topic_model.visualize_topics()
# #topic_model.visualize_heatmap()
topic_model.visualize_barchart()

In [ ]:
topic_model = BERTopic.load(MODEL_PATH + "bertopic_model_mvb_new") #load topic model
#topic_model.visualize_barchart()

In [ ]:
topic_model.get_topics()

In [ ]:
dft = topic_model.get_topic_info()
dft

In [ ]:
#CODE CUSTOMIZED VISUALIZE_BARCHART FIGURE

import itertools
import numpy as np
from typing import List

import plotly.graph_objects as go
from plotly.subplots import make_subplots


def visualize_barchart(topic_model,
                       topics: List[int] = None,
                       top_n_topics: int = 8,
                       n_words: int = 10,
                       custom_labels: bool = False,
                       title: str = "Topic Word Scores",
                       width: int = 250,
                       height: int = 250) -> go.Figure:
    """ Visualize a barchart of selected topics
    Arguments:
        topic_model: A fitted BERTopic instance.
        topics: A selection of topics to visualize.
        top_n_topics: Only select the top n most frequent topics.
        n_words: Number of words to show in a topic
        custom_labels: Whether to use custom topic labels that were defined using 
                       `topic_model.set_topic_labels`.
        title: Title of the plot.
        width: The width of each figure.
        height: The height of each figure.
    Returns:
        fig: A plotly figure
    Examples:
    To visualize the barchart of selected topics
    simply run:
    ```python
    topic_model.visualize_barchart()
    ```
    Or if you want to save the resulting figure:
    ```python
    fig = topic_model.visualize_barchart()
    fig.write_html("path/to/file.html")
    ```
    <iframe src="../../getting_started/visualization/bar_chart.html"
    style="width:1100px; height: 660px; border: 0px;""></iframe>
    """
    colors = itertools.cycle(["#D55E00", "#0072B2", "#CC79A7", "#E69F00", "#56B4E9", "#009E73", "#F0E442"])

    # Select topics based on top_n and topics args
    freq_df = topic_model.get_topic_freq()
    freq_df = freq_df.loc[freq_df.Topic != -1, :]
    if topics is not None:
        topics = list(topics)
    elif top_n_topics is not None:
        topics = sorted(freq_df.Topic.to_list()[:top_n_topics])
    else:
        topics = sorted(freq_df.Topic.to_list()[0:6])

    # Initialize figure
    if topic_model.custom_labels is not None and custom_labels:
        subplot_titles = [topic_model.custom_labels[topic + topic_model._outliers] for topic in topics]
    else:
        subplot_titles = [f"Topic {topic}" for topic in topics]
    columns = 3
    rows = int(np.ceil(len(topics) / columns))
    fig = make_subplots(rows=rows,
                        cols=columns,
                        shared_xaxes=False,
                        horizontal_spacing=.1,
                        vertical_spacing=.4 / rows if rows > 1 else 0,
                        subplot_titles=subplot_titles)

    # Add barchart for each topic
    row = 1
    column = 1
    for topic in topics:
        words = [word + "  " for word, _ in topic_model.get_topic(topic)][:n_words][::-1]
        scores = [score for _, score in topic_model.get_topic(topic)][:n_words][::-1]

        fig.add_trace(
            go.Bar(x=scores,
                   y=words,
                   orientation='h',
                   marker_color=next(colors)),
            row=row, col=column)

        if column == columns:
            column = 1
            row += 1
        else:
            column += 1

    # Stylize graph
    fig.update_layout(
        template="plotly_white",
        showlegend=False,
        title={
            'text': f"<b>{title}",
            'x': .5,
            'xanchor': 'center',
            'yanchor': 'top',
            'font': dict(
                size=22,
                color="Black")
        },
        width=width*4,
        height=height*rows if rows > 1 else height * 1.3,
        hoverlabel=dict(
            bgcolor="white",
            font_size=16,
            font_family="Rockwell"
        ),
    )

    fig.update_xaxes(showgrid=True)
    fig.update_yaxes(showgrid=True)

    return fig

visualize_barchart(topic_model,topics=[25,26,27,28,29,30],custom_labels=True)

In [ ]:
#Analyze topic model
# topic_model.get_topic(3)#get words of certain topic
# topic_model.find_topics("vehicle")#get topics to which certain word belongs
# topic_model.transform([docs[103]])
# topic_model.reduce_topics(docs, topics, nr_topics=25)
# topic_model.get_representative_docs()
# topic_model.merge_topics(docs, topics, [-1,13]) 
# #Topics to merge: [1,2] or [[1, 2], [3, 4]]
# #NB topics should be list of topic numbers for each doc in docs